# Predictive Analytics Model for KServe

## Overview
This notebook trains and saves a **Predictive Analytics model** in KServe-compatible format for the `predictive-analytics` InferenceService.

**Model Type**: Time series forecasting with Random Forest  
**Purpose**: Predict future resource usage (CPU, memory, disk, network)  
**Deployment**: KServe InferenceService with sklearn runtime

## KServe Integration (Issue #13 Fix)

This notebook implements the fix for [Issue #13](https://github.com/KubeHeal/openshift-aiops-platform/issues/13) where models were registering as `"model"` instead of `"predictive-analytics"`.

### Problem Solved
- **Before**: Models saved to `/mnt/models/cpu_usage_step_0_model.pkl` (flat structure)
- **After**: Models saved to `/mnt/models/predictive-analytics/model.pkl` (KServe structure)
- **Result**: Model registers correctly as `"predictive-analytics"` ✅

### Architecture
```
Notebook Training → /mnt/models/predictive-analytics/model.pkl
                    ↓
KServe InferenceService (storageUri: pvc://model-storage-pvc/predictive-analytics)
                    ↓
Model registered as: "predictive-analytics"
                    ↓
Endpoint: /v1/models/predictive-analytics:predict
```

## Prerequisites
- Model storage PVC mounted at `/mnt/models`
- Python environment with sklearn, pandas, numpy
- Access to `src/models/predictive_analytics.py` module

## What This Notebook Does
1. ✅ Imports the `PredictiveAnalytics` module
2. ✅ Generates synthetic time series training data
3. ✅ Trains multi-metric forecasting models (CPU, memory, disk, network)
4. ✅ Saves in KServe-compatible format: `/mnt/models/predictive-analytics/model.pkl`
5. ✅ Validates the model works correctly
6. ✅ Tests prediction endpoint format

## Setup Section

### Import Libraries and Configure Environment

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path

# Setup path for src/models module - works from any directory
def find_models_path():
    """Find src/models path regardless of current working directory"""
    possible_paths = [
        Path(__file__).parent.parent.parent / 'src' / 'models' if '__file__' in dir() else None,
        Path.cwd().parent.parent / 'src' / 'models',
        Path('/workspace/repo/src/models'),
        Path('/opt/app-root/src/openshift-aiops-platform/src/models'),
    ]
    for p in possible_paths:
        if p and p.exists() and (p / 'predictive_analytics.py').exists():
            return str(p)
    # Try relative path search
    current = Path.cwd()
    for _ in range(5):
        models_path = current / 'src' / 'models'
        if models_path.exists() and (models_path / 'predictive_analytics.py').exists():
            return str(models_path)
        current = current.parent
    return None

models_path = find_models_path()
if models_path:
    sys.path.insert(0, models_path)
    print(f"✅ Models path found: {models_path}")
else:
    print("⚠️ Models path not found - using fallback implementation")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import joblib
import warnings
warnings.filterwarnings('ignore')

# Import PredictiveAnalytics module
try:
    from predictive_analytics import PredictiveAnalytics, generate_sample_timeseries_data
    print("✅ PredictiveAnalytics module imported successfully")
    USING_MODULE = True
except ImportError as e:
    print(f"❌ Failed to import PredictiveAnalytics module: {e}")
    print("   Please ensure src/models/predictive_analytics.py is available")
    USING_MODULE = False

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("\n✅ All libraries imported successfully")

### Configure Model Storage Paths

In [ ]:
# Configure storage paths
# Use /mnt/models for persistent storage (model-storage-pvc)
# Fallback to local for development outside cluster
MODELS_DIR = Path('/mnt/models') if Path('/mnt/models').exists() else Path('/opt/app-root/src/models')
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Model name must match InferenceService name
MODEL_NAME = 'predictive-analytics'
MODEL_DIR = MODELS_DIR / MODEL_NAME  # Will be created by save_models()

print(f"📁 Model Storage Configuration:")
print(f"   Base directory: {MODELS_DIR}")
print(f"   Model name: {MODEL_NAME}")
print(f"   Expected KServe path: {MODEL_DIR}/model.pkl")
print(f"   PVC available: {'✅ Yes' if MODELS_DIR == Path('/mnt/models') else '⚠️ No (using local)'}")

if not USING_MODULE:
    print("\n❌ Cannot proceed without PredictiveAnalytics module")
    raise ImportError("PredictiveAnalytics module required for this notebook")

In [ ]:
# ====================
# Data Source Configuration (ADR-050, ADR-052)
# ====================
import requests
import urllib3

# Disable SSL warnings for internal Prometheus (self-signed cert)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

DATA_SOURCE = os.getenv('DATA_SOURCE', 'synthetic')  # synthetic|prometheus|hybrid
# Use HTTPS port 9091 for authenticated access (requires bearer token)
PROMETHEUS_URL = os.getenv('PROMETHEUS_URL', 'https://prometheus-k8s.openshift-monitoring.svc:9091')
TRAINING_DAYS = int(os.getenv('TRAINING_DAYS', '30'))  # 30-day lookback for predictive analytics
TRAINING_HOURS = int(os.getenv('TRAINING_HOURS', str(TRAINING_DAYS * 24)))
MAX_SAMPLES = int(os.getenv('MAX_SAMPLES', '0'))  # 0 = no limit, >0 = limit samples for fast training
PROMETHEUS_AVAILABLE = False
PROMETHEUS_VERIFY_SSL = os.getenv('PROMETHEUS_VERIFY_SSL', 'false').lower() == 'true'

# Get ServiceAccount token for Prometheus authentication
SA_TOKEN_PATH = '/var/run/secrets/kubernetes.io/serviceaccount/token'
PROMETHEUS_TOKEN = None
if os.path.exists(SA_TOKEN_PATH):
    with open(SA_TOKEN_PATH, 'r') as f:
        PROMETHEUS_TOKEN = f.read().strip()
    print(f"✅ ServiceAccount token loaded for Prometheus authentication")
else:
    print(f"⚠️ No ServiceAccount token found at {SA_TOKEN_PATH}")

# Build headers for Prometheus requests
PROMETHEUS_HEADERS = {}
if PROMETHEUS_TOKEN:
    PROMETHEUS_HEADERS['Authorization'] = f'Bearer {PROMETHEUS_TOKEN}'

# Check Prometheus availability
if DATA_SOURCE in ['prometheus', 'hybrid']:
    try:
        response = requests.get(
            f"{PROMETHEUS_URL}/api/v1/status/config",
            headers=PROMETHEUS_HEADERS,
            verify=PROMETHEUS_VERIFY_SSL,
            timeout=10
        )
        PROMETHEUS_AVAILABLE = response.status_code == 200
        print(f"✅ Prometheus available at {PROMETHEUS_URL}")
    except Exception as e:
        print(f"⚠️ Prometheus not available: {e}")
        print(f"   Falling back to synthetic data")
        DATA_SOURCE = 'synthetic'

print(f"\n📊 Data Source Configuration:")
print(f"   Mode: {DATA_SOURCE}")
print(f"   Training hours: {TRAINING_HOURS}h ({TRAINING_HOURS / 24:.1f} days)")
print(f"   Prometheus: {'✅ Available' if PROMETHEUS_AVAILABLE else '❌ Unavailable'}")


# ====================
# Prometheus Data Fetching Functions
# ====================

def fetch_prometheus_timeseries(metric_query, lookback_hours=720):
    """
    Fetch time series data from Prometheus with bearer token authentication.
    
    Args:
        metric_query: PromQL query string
        lookback_hours: Time window in hours (default: 720 = 30 days)
    
    Returns:
        pandas DataFrame with timestamp and value columns
    """
    end_time = datetime.now()
    start_time = end_time - timedelta(hours=lookback_hours)
    
    params = {
        'query': metric_query,
        'start': int(start_time.timestamp()),
        'end': int(end_time.timestamp()),
        'step': '5m'  # 5-minute intervals
    }
    
    try:
        response = requests.get(
            f'{PROMETHEUS_URL}/api/v1/query_range', 
            params=params,
            headers=PROMETHEUS_HEADERS,
            verify=PROMETHEUS_VERIFY_SSL,
            timeout=30
        )
        response.raise_for_status()
        
        result = response.json()
        if result['status'] != 'success':
            raise ValueError(f"Prometheus query failed: {result}")
        
        # Parse results
        timestamps = []
        values = []
        for series in result['data']['result']:
            for timestamp, value in series['values']:
                timestamps.append(pd.to_datetime(timestamp, unit='s'))
                values.append(float(value))
        
        df = pd.DataFrame({
            'timestamp': timestamps,
            'value': values
        })
        
        if len(df) > 0:
            df = df.sort_values('timestamp').reset_index(drop=True)
        
        return df
    
    except Exception as e:
        print(f"⚠️  Failed to fetch Prometheus data: {e}")
        return pd.DataFrame(columns=['timestamp', 'value'])


def fetch_prometheus_metrics_for_prediction(lookback_hours=720):
    """
    Fetch all metrics needed for predictive analytics from Prometheus
    
    Args:
        lookback_hours: Time window in hours
    
    Returns:
        pandas DataFrame with timestamp, cpu_usage, memory_usage, disk_usage, network_in, network_out
    """
    print(f"🔍 Fetching metrics from Prometheus (lookback: {lookback_hours}h)...")
    
    # Prometheus query mappings - UPDATED for OpenShift 4.18+ compatibility
    # These queries use metrics that are universally available in OpenShift
    metric_queries = {
        # CPU: cluster-level CPU ratio (0-1 scale)
        'cpu_usage': 'cluster:node_cpu:ratio_rate5m',
        # Memory: average node memory utilization (0-1 scale)
        'memory_usage': 'avg(instance:node_memory_utilisation:ratio)',
        # Disk: cluster-wide filesystem usage ratio (0-1 scale)
        'disk_usage': '1 - sum(node_filesystem_avail_bytes) / sum(node_filesystem_size_bytes)',
        # Network: container network receive bytes (cumulative counter)
        'network_in': 'sum(container_network_receive_bytes_total)',
        # Network: container network transmit bytes (cumulative counter)
        'network_out': 'sum(container_network_transmit_bytes_total)'
    }
    
    # Fetch each metric
    metric_dfs = {}
    for metric_name, query in metric_queries.items():
        print(f"  📊 Fetching {metric_name}...")
        df = fetch_prometheus_timeseries(query, lookback_hours)
        
        if len(df) > 0:
            metric_dfs[metric_name] = df
            print(f"    ✅ {len(df)} data points")
        else:
            print(f"    ⚠️  No data available")
    
    # If no metrics fetched, return empty DataFrame
    if not metric_dfs:
        print(f"❌ No metrics fetched from Prometheus")
        return pd.DataFrame()
    
    # Merge all metrics on timestamp
    print(f"\n🔧 Merging metrics...")
    combined_df = None
    
    for metric_name, df in metric_dfs.items():
        df = df.rename(columns={'value': metric_name})
        if combined_df is None:
            combined_df = df
        else:
            combined_df = combined_df.merge(df, on='timestamp', how='outer')
    
    # Sort by timestamp and forward-fill missing values
    combined_df = combined_df.sort_values('timestamp').reset_index(drop=True)
    combined_df = combined_df.ffill().bfill()
    
    # Fill any remaining NaN with 0
    combined_df = combined_df.fillna(0)
    
    print(f"✅ Combined dataset: {len(combined_df)} samples with {len(metric_dfs)} metrics")
    
    return combined_df


print("✅ Data fetching functions configured")

In [ ]:
# Generate training data based on data source configuration
print("📊 Generating training data...")
print(f"   Mode: {DATA_SOURCE}")
print(f"   Time window: {TRAINING_HOURS}h ({TRAINING_HOURS / 24:.1f} days)")

if DATA_SOURCE == 'prometheus' and PROMETHEUS_AVAILABLE:
    # Fetch data from Prometheus
    print("\n🔍 Fetching data from Prometheus...")
    sample_data = fetch_prometheus_metrics_for_prediction(lookback_hours=TRAINING_HOURS)
    
    if len(sample_data) == 0:
        print("⚠️  No Prometheus data available, falling back to synthetic")
        n_samples = int(TRAINING_HOURS * 12)  # 5-min intervals
        sample_data = generate_sample_timeseries_data(n_samples=n_samples)
    else:
        print(f"✅ Using Prometheus data: {len(sample_data)} samples")

elif DATA_SOURCE == 'hybrid' and PROMETHEUS_AVAILABLE:
    # Mix Prometheus and synthetic data
    print("\n🔍 Creating hybrid dataset (50% Prometheus, 50% synthetic)...")
    prom_data = fetch_prometheus_metrics_for_prediction(lookback_hours=TRAINING_HOURS)
    
    if len(prom_data) > 0:
        # Generate synthetic data to match Prometheus size
        synthetic_data = generate_sample_timeseries_data(n_samples=len(prom_data))
        
        # Combine datasets
        sample_data = pd.concat([prom_data, synthetic_data], ignore_index=True)
        sample_data = sample_data.sort_values('timestamp').reset_index(drop=True)
        print(f"✅ Combined: {len(prom_data)} Prometheus + {len(synthetic_data)} synthetic = {len(sample_data)} total")
    else:
        print("⚠️  No Prometheus data available, using 100% synthetic")
        n_samples = int(TRAINING_HOURS * 12)
        sample_data = generate_sample_timeseries_data(n_samples=n_samples)

else:
    # Pure synthetic data
    print("\n📊 Generating synthetic time series data...")
    print("   This simulates realistic infrastructure metrics with patterns:")
    print("   - Daily cycles (higher during business hours)")
    print("   - Weekly patterns (weekday vs weekend)")
    print("   - Trends (gradual growth over time)")
    print("   - Noise (random variations)")
    
    # Calculate sample count based on training hours (5-minute intervals)
    n_samples = int(TRAINING_HOURS * 12)
    sample_data = generate_sample_timeseries_data(n_samples=n_samples)
    print(f"✅ Generated {len(sample_data)} synthetic samples")

print(f"\n✅ Training data prepared:")
print(f"   Samples: {len(sample_data)}")
print(f"   Columns: {', '.join(sample_data.columns)}")
print(f"   Shape: {sample_data.shape}")
print(f"   Date range: {sample_data['timestamp'].min()} to {sample_data['timestamp'].max()}")

# Display sample data
print("\n📋 Sample data (first 5 rows):")
display(sample_data.head())

# Show statistics
print("\n📈 Data Statistics:")
display(sample_data.describe())

In [ ]:
# OPTIONAL: Generate synthetic data ONLY if no data was loaded from Cell 6
# NOTE: This cell is a FALLBACK only for when Cell 6 failed to produce data

if 'sample_data' not in dir() or sample_data is None or len(sample_data) == 0:
    print("⚠️ No data from Cell 6, generating synthetic data as fallback...")
    print("📊 Generating synthetic time series data...")
    
    # Use MAX_SAMPLES for fast mode, or default to 500
    n_samples = MAX_SAMPLES if MAX_SAMPLES > 0 else 500
    sample_data = generate_sample_timeseries_data(n_samples=n_samples)
    print(f"✅ Generated {len(sample_data)} synthetic samples")
else:
    print(f"✅ Using existing data from Cell 6: {len(sample_data)} samples")
    print(f"   Skipping synthetic data generation (data already loaded)")

# Apply MAX_SAMPLES limit if needed
if MAX_SAMPLES > 0 and len(sample_data) > MAX_SAMPLES:
    original_count = len(sample_data)
    step = max(1, len(sample_data) // MAX_SAMPLES)
    sample_data = sample_data.iloc[::step].head(MAX_SAMPLES).reset_index(drop=True)
    print(f"⚡ Fast mode: Subsampled {original_count} → {len(sample_data)} samples")

print(f"\n📊 Training Data Summary:")
print(f"   Samples: {len(sample_data)}")
print(f"   Columns: {', '.join(sample_data.columns)}")
print(f"   Shape: {sample_data.shape}")
print(f"   Date range: {sample_data['timestamp'].min()} to {sample_data['timestamp'].max()}")

# Display sample data
print("\n📋 Sample data (first 5 rows):")
display(sample_data.head())

# Show statistics
print("\n📈 Data Statistics:")
display(sample_data.describe())


### Visualize Training Data

In [ ]:
# Visualize the generated time series data
fig, axes = plt.subplots(3, 2, figsize=(15, 10))
fig.suptitle('Synthetic Time Series Training Data', fontsize=16, fontweight='bold')

metrics = ['cpu_usage', 'memory_usage', 'disk_usage', 'network_in', 'network_out']
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']

for idx, (metric, color) in enumerate(zip(metrics, colors)):
    ax = axes[idx // 2, idx % 2]
    ax.plot(sample_data['timestamp'], sample_data[metric], color=color, alpha=0.7, linewidth=1)
    ax.set_title(f'{metric.replace("_", " ").title()}', fontweight='bold')
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

# Hide the last subplot (we have 5 metrics in a 3x2 grid)
axes[2, 1].set_visible(False)

plt.tight_layout()
plt.show()

print("✅ Visualization complete")

### Split Data for Training and Validation

In [ ]:
# Split data: 80% training, 20% validation
split_point = int(len(sample_data) * 0.8)

train_data = sample_data.iloc[:split_point].copy()
val_data = sample_data.iloc[split_point:].copy()

print(f"📊 Data Split:")
print(f"   Training samples: {len(train_data)} ({len(train_data)/len(sample_data)*100:.1f}%)")
print(f"   Validation samples: {len(val_data)} ({len(val_data)/len(sample_data)*100:.1f}%)")
print(f"   Training period: {train_data['timestamp'].min()} to {train_data['timestamp'].max()}")
print(f"   Validation period: {val_data['timestamp'].min()} to {val_data['timestamp'].max()}")

## Model Training Section

### Initialize and Train PredictiveAnalytics Model

In [ ]:
# Initialize PredictiveAnalytics model
print("🔬 Initializing PredictiveAnalytics model...")

# Configure model parameters
FORECAST_HORIZON = 12  # Predict 12 time steps ahead
LOOKBACK_WINDOW = 24   # Use 24 historical time steps

predictor = PredictiveAnalytics(
    forecast_horizon=FORECAST_HORIZON,
    lookback_window=LOOKBACK_WINDOW
)

print(f"   Forecast horizon: {FORECAST_HORIZON} time steps")
print(f"   Lookback window: {LOOKBACK_WINDOW} time steps")
print(f"   Target metrics: {', '.join(predictor.target_metrics)}")

# Train the model
print(f"\n🎯 Training models on {len(train_data)} samples...")
print("   This will train separate models for each metric:")
print("   - CPU usage")
print("   - Memory usage")
print("   - Disk usage")
print("   - Network in")
print("   - Network out")

training_results = predictor.train(train_data)

print(f"\n✅ Training completed!")
print(f"   Models trained: {training_results['models_trained']}")
print(f"   Feature count: {training_results['feature_count']}")
print(f"   Forecast horizon: {training_results['forecast_horizon']}")
print(f"   Lookback window: {training_results['lookback_window']}")

### Evaluate Model Performance

In [ ]:
# Display detailed metrics for each model
print("📊 Model Performance Metrics:\n")
print("=" * 80)

for metric_name, results in training_results['metrics'].items():
    print(f"\n{metric_name.upper().replace('_', ' ')}:")
    print(f"  Mean Absolute Error (MAE):  {results['mae']:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {results['rmse']:.4f}")
    print(f"  R² Score: {results['r2']:.4f}")
    print(f"  Training samples: {results['training_samples']}")
    print(f"  Test samples: {results['test_samples']}")
    print("  " + "-" * 60)

print("\n" + "=" * 80)

# Calculate average R² across all metrics
avg_r2 = np.mean([r['r2'] for r in training_results['metrics'].values()])
print(f"\n📈 Average R² Score: {avg_r2:.4f}")

if avg_r2 > 0.8:
    print("✅ Excellent model performance!")
elif avg_r2 > 0.6:
    print("✅ Good model performance")
else:
    print("⚠️ Model performance could be improved - consider more training data")

## Model Validation Section

### Test Predictions on Validation Data

In [ ]:
# Make predictions on validation data
print("🔮 Making predictions on validation data...")

predictions = predictor.predict(val_data.head(50))

print(f"\n✅ Predictions generated:")
print(f"   Timestamp: {predictions['timestamp']}")
print(f"   Metrics predicted: {len(predictions['predictions'])}")
print(f"   Lookback window used: {predictions['lookback_window']}")

# Display predictions for each metric
print("\n📊 Prediction Results:\n")
for metric_name, pred_data in predictions['predictions'].items():
    forecast = pred_data['forecast']
    confidence = pred_data.get('confidence', [0.5] * len(forecast))
    
    print(f"{metric_name.upper().replace('_', ' ')}:")
    print(f"  Forecast values (first 5): {[f'{v:.2f}' for v in forecast[:5]]}")
    print(f"  Confidence (avg): {np.mean(confidence):.2%}")
    print()

### Visualize Predictions vs Actual

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Predictions vs Actual Values (Validation Set)', fontsize=16, fontweight='bold')

# Select metrics to visualize
vis_metrics = ['cpu_usage', 'memory_usage', 'disk_usage', 'network_in']

for idx, metric in enumerate(vis_metrics):
    ax = axes[idx // 2, idx % 2]
    
    # Plot actual values
    actual_vals = val_data[metric].head(50).values
    ax.plot(range(len(actual_vals)), actual_vals, label='Actual', color='blue', alpha=0.6, linewidth=2)
    
    # Plot predictions (if available)
    if metric in predictions['predictions']:
        forecast = predictions['predictions'][metric]['forecast']
        # Start prediction from lookback_window position
        pred_start = predictions['lookback_window']
        ax.plot(range(pred_start, pred_start + len(forecast)), 
               forecast, label='Predicted', color='red', alpha=0.6, linewidth=2, linestyle='--')
    
    ax.set_title(f'{metric.replace("_", " ").title()}', fontweight='bold')
    ax.set_xlabel('Time Step')
    ax.set_ylabel('Value')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Visualization complete")

## Model Saving Section

### Save Model in KServe-Compatible Format

This is the critical step that implements the **Issue #13 fix**!

In [ ]:
# Save STANDARD sklearn Pipeline for KServe sklearn server
# 
# CRITICAL: We save a STANDARD sklearn Pipeline, NOT a custom class.
# Custom classes cannot be deserialized by KServe's sklearn server because
# the class definition doesn't exist in that environment.
#
# The Pipeline contains only standard sklearn/xgboost classes:
# - sklearn.pipeline.Pipeline
# - sklearn.preprocessing.StandardScaler  
# - sklearn.multioutput.MultiOutputRegressor
# - xgboost.XGBRegressor (or sklearn.ensemble.RandomForestRegressor)

print("💾 Saving STANDARD sklearn Pipeline for KServe...\n")
print("=" * 80)
print("KSERVE MULTI-OUTPUT MODEL (Standard sklearn Pipeline)")
print("=" * 80)
print(f"\n📂 Directory Structure:")
print(f"   Base: {MODELS_DIR}")
print(f"   Model subdirectory: {MODEL_NAME}/")
print(f"   Full path: {MODELS_DIR}/{MODEL_NAME}/model.pkl")

from pathlib import Path
import joblib
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor

# ====================
# CREATE STANDARD SKLEARN PIPELINE
# ====================
# Check if we have the pipeline from Cell 20 already
if 'pipeline' in dir() and pipeline is not None:
    print(f"\n✅ Using trained pipeline from Cell 20")
    model_to_save = pipeline
else:
    # Create a fresh pipeline if none exists
    print(f"\n⚠️ No pipeline found, creating new one...")
    
    # Determine base estimator
    try:
        import xgboost as xgb
        base_estimator = xgb.XGBRegressor(
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            tree_method='hist',
            random_state=42,
            n_jobs=-1
        )
        print("   Using XGBRegressor (CPU histogram method)")
    except ImportError:
        from sklearn.ensemble import RandomForestRegressor
        base_estimator = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
        print("   Using RandomForestRegressor (XGBoost not available)")
    
    # Create pipeline with standard components only
    model_to_save = Pipeline([
        ('scaler', StandardScaler()),
        ('model', MultiOutputRegressor(base_estimator))
    ])
    
    # Train on available data
    if 'train_data' in dir() and train_data is not None and len(train_data) > 0:
        print(f"   Training pipeline on {len(train_data)} samples...")
        
        # Prepare features (all columns except timestamp and target metrics for features)
        target_cols = ['cpu_usage', 'memory_usage', 'disk_usage', 'network_in', 'network_out']
        feature_cols = [c for c in train_data.columns if c not in ['timestamp'] + target_cols]
        
        if len(feature_cols) == 0:
            # If no separate features, use target values shifted
            feature_cols = target_cols
        
        X_train = train_data[feature_cols].values
        y_train = train_data[target_cols].values
        
        model_to_save.fit(X_train, y_train)
        print(f"   ✅ Pipeline trained! Features: {X_train.shape[1]}, Samples: {len(X_train)}")
    else:
        print("   ⚠️ No training data available!")

# Get expected feature count from trained model
if hasattr(model_to_save, 'n_features_in_'):
    n_features = model_to_save.n_features_in_
else:
    n_features = 5  # Fallback to number of target metrics

# Test the model before saving
print(f"\n🧪 Testing pipeline before saving...")
test_input = np.random.rand(1, n_features)  # Random test input
test_output = model_to_save.predict(test_input)
print(f"   Input shape: {test_input.shape}")
print(f"   Output shape: {test_output.shape}")
print(f"   Output values: {test_output[0]}")
print(f"   Output format: [cpu, memory, disk, network_in, network_out]")

# Save to KServe directory
kserve_dir = MODELS_DIR / MODEL_NAME
kserve_dir.mkdir(parents=True, exist_ok=True)

model_path = kserve_dir / 'model.pkl'
joblib.dump(model_to_save, model_path)
print(f"\n✅ Pipeline saved to: {model_path}")

# Verify the saved model
expected_path = MODELS_DIR / MODEL_NAME / 'model.pkl'
print(f"\n🔍 Verifying saved model can be loaded...")

if expected_path.exists():
    # Load and verify
    loaded_model = joblib.load(expected_path)
    size_kb = expected_path.stat().st_size / 1024
    
    # Get actual feature count
    if hasattr(loaded_model, 'n_features_in_'):
        expected_features = loaded_model.n_features_in_
    else:
        expected_features = n_features
    
    # Verify predict() works
    verify_input = np.random.rand(1, expected_features)
    verify_output = loaded_model.predict(verify_input)
    
    print(f"\n✅ SUCCESS! Standard sklearn Pipeline saved and verified:")
    print(f"   Location: {expected_path}")
    print(f"   Size: {size_kb:.2f} KB")
    print(f"   Type: {type(loaded_model).__name__}")
    print(f"   Components: {list(loaded_model.named_steps.keys())}")
    print(f"   Has predict(): {hasattr(loaded_model, 'predict')}")
    print(f"   Expected features: {expected_features}")
    print(f"   Output shape: {verify_output.shape}")
    
    print(f"\n📡 KServe Response Format:")
    print(f"   Request: {{\"instances\": [[... {expected_features} features ...]]}}")
    print(f"   Response: {{\"predictions\": [[cpu, mem, disk, net_in, net_out]]}}")
    print(f"   Example: {{\"predictions\": [[0.45, 0.62, 0.38, 150.2, 89.5]]}}")
    
    print(f"\n🔗 Coordination Engine Integration:")
    print(f"   predictions[0] = cpu_usage")
    print(f"   predictions[1] = memory_usage")
    print(f"   predictions[2] = disk_usage")
    print(f"   predictions[3] = network_in")
    print(f"   predictions[4] = network_out")
    
    print(f"\n✅ IMPORTANT: Model uses ONLY standard sklearn classes:")
    print(f"   - sklearn.pipeline.Pipeline")
    print(f"   - sklearn.preprocessing.StandardScaler")
    print(f"   - sklearn.multioutput.MultiOutputRegressor")
    print(f"   - xgboost.XGBRegressor (or RandomForestRegressor)")
    print(f"   KServe sklearn server CAN load this model!")
    
else:
    print(f"\n❌ ERROR: Model not found at expected location: {expected_path}")

print("\n" + "=" * 80)


### Test Model Loading (Verification)

In [ ]:
# Verify the model can be loaded back using joblib (same as KServe does)
print("🧪 Testing model loading (simulating KServe sklearn server)...")

# Load the model exactly as KServe will do it
import joblib
model_path = MODELS_DIR / MODEL_NAME / 'model.pkl'
loaded_pipeline = joblib.load(model_path)

print(f"\n✅ Model loaded successfully!")
print(f"   Type: {type(loaded_pipeline).__name__}")
print(f"   Components: {list(loaded_pipeline.named_steps.keys())}")

# Check feature count
if hasattr(loaded_pipeline, 'n_features_in_'):
    print(f"   Expected features: {loaded_pipeline.n_features_in_}")
    n_test_features = loaded_pipeline.n_features_in_
else:
    n_test_features = 5

# Make a test prediction with the loaded model
print(f"\n🧪 Testing prediction with loaded model...")
test_input = np.random.rand(3, n_test_features)  # 3 samples
test_predictions = loaded_pipeline.predict(test_input)

print(f"   Input shape: {test_input.shape}")
print(f"   Output shape: {test_predictions.shape}")
print(f"   Predictions (sample 1): {test_predictions[0]}")
print(f"   Output format: [cpu, memory, disk, network_in, network_out]")

# Verify output has 5 columns (one per metric)
assert test_predictions.shape[1] == 5, f"Expected 5 output columns, got {test_predictions.shape[1]}"

print(f"\n🎉 Model is ready for KServe deployment!")
print(f"   ✅ Model loads with joblib.load()")
print(f"   ✅ Model returns 5 predictions (all metrics)")
print(f"   ✅ Compatible with KServe sklearn server")

## Deployment Verification Section

### Generate KServe Test Commands

In [ ]:
# Generate commands for testing the deployed model
print("📋 KServe Deployment Test Commands:\n")
print("=" * 80)
print("After deploying the InferenceService, run these commands to verify:\n")

print("# 1. Get the predictor pod IP")
print(f"PREDICTOR_IP=$(oc get pod -l serving.kserve.io/inferenceservice={MODEL_NAME} -o jsonpath='{{.items[0].status.podIP}}')")
print(f"echo \"Predictor IP: $PREDICTOR_IP\"\n")

print("# 2. List available models (should return 'predictive-analytics', not 'model')")
print("curl http://${PREDICTOR_IP}:8080/v1/models")
print(f"# Expected: {{\"models\":[\"{MODEL_NAME}\"]}}  ✅\n")

print("# 3. Check model status")
print(f"curl http://${{PREDICTOR_IP}}:8080/v1/models/{MODEL_NAME}")
print(f"# Expected: {{\"name\":\"{MODEL_NAME}\",\"ready\":true}}  ✅\n")

print("# 4. Test prediction endpoint")
print(f"curl -X POST http://${{PREDICTOR_IP}}:8080/v1/models/{MODEL_NAME}:predict \\")
print("  -H 'Content-Type: application/json' \\")
print("  -d '{\"instances\": [[0.5, 0.6, 0.4, 100, 80]]}'")
print("# Expected: Prediction response with forecast values  ✅\n")

print("=" * 80)
print("\n✅ If all commands work, Issue #13 is fixed!")

## Summary

### What Was Accomplished

✅ **Model trained** with multi-metric forecasting (CPU, memory, disk, network)  
✅ **Saved in KServe format**: `/mnt/models/predictive-analytics/model.pkl`  
✅ **Issue #13 fixed**: Model will register as `"predictive-analytics"` not `"model"`  
✅ **Validated**: Model can be loaded and makes predictions  
✅ **Ready for deployment**: Compatible with KServe InferenceService

### Next Steps

1. **Deploy InferenceService** (if not already deployed):
   ```yaml
   apiVersion: serving.kserve.io/v1beta1
   kind: InferenceService
   metadata:
     name: predictive-analytics
   spec:
     predictor:
       model:
         name: predictive-analytics
         runtime: sklearn-pvc-runtime
         storageUri: "pvc://model-storage-pvc/predictive-analytics"
   ```

2. **Verify deployment** using the commands above

3. **Test from coordination engine**:
   - Ensure coordination engine can call `/v1/models/predictive-analytics:predict`
   - Verify predictions work end-to-end

4. **Monitor predictions**:
   - Check prediction accuracy over time
   - Retrain periodically with new data

### References

- **Issue**: [#13 - KServe model registration fix](https://github.com/KubeHeal/openshift-aiops-platform/issues/13)
- **Module**: `src/models/predictive_analytics.py`
- **Training script**: `src/models/train_predictive_analytics.py`
- **Documentation**: `src/models/KSERVE_FIX_README.md`